In [26]:
import numpy as np

from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python.platform import flags
import parser 

# FLAGS = flags.FLAGS

# flags.DEFINE_string('dataset_folder', './ml-100k/', 'Path to Movielens 100k/1m dataset')
# flags.DEFINE_integer('bsize', 32, 'Batch size')
# flags.DEFINE_integer('num_iters', 500, 'Number of iterations')
# flags.DEFINE_integer('show_every', 10, 'Show trainign and testing accuracy after every X iterations')
# flags.DEFINE_float('lr', 0.01, 'Learning rate')
# flags.DEFINE_float('reg_lambda', 0.01, "Lambda for regularization")

# Training Parameters
# learning_rate = FLAGS.lr
# num_iters = FLAGS.num_iters
# batch_size = FLAGS.bsize
# lamb = FLAGS.reg_lambda
# dd = FLAGS.dataset_folder
# show_every = FLAGS.show_every

learning_rate = 0.01
num_iters = 20000
batch_size = 16
lamb = 0.1
dd = './ml-100k/'
show_every = 100


# Don't hog GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [27]:
def nmae(y, y_, minr=1, maxr=5):
    flaty = y.flatten()
    flaty_ = y_.flatten()
    relevant_count = flaty.shape[0]
    return np.sum(np.absolute(flaty-flaty_))/(relevant_count * (maxr-minr))

In [28]:
def autorec(num_input, num_hidden, data_generator, test_data):
    X = tf.placeholder("float", [None, R.shape[1]])
    
    weights = {
        'encoder': tf.Variable(tf.random_normal([num_input, num_hidden])),
        'decoder': tf.Variable(tf.random_normal([num_hidden, num_input])),
    }
    biases = {
        'encoder': tf.Variable(tf.random_normal([num_hidden])),
        'decoder': tf.Variable(tf.random_normal([num_input])),
    }

    def encoder(x):
        layer = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder']), biases['encoder']))
        mask = tf.cast(tf.not_equal(tf.constant(0, dtype=tf.float32), x), tf.float32)
        return layer, mask

    def decoder(x, mask):
        layer = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder']), biases['decoder']))
        masked_backfunc = tf.multiply(layer, mask)
        # Trick to allow back-gradient propagataion for only part of input nodes
        back_masked_layer = masked_backfunc + tf.stop_gradient(layer - masked_backfunc) 
        return layer
    
    encoder_op, mask = encoder(X)
    decoder_op = decoder(encoder_op, mask)

    y_pred = decoder_op
    y_true = X
    
    # Normal MSE loss for autoencoder:
    loss = tf.reduce_mean(tf.where(y_true==0, tf.zeros_like(y_true), tf.pow(y_true - y_pred, 2)))
    # Regularization term:
    loss += (lamb/2) * (tf.nn.l2_loss(weights['encoder']) + tf.nn.l2_loss(weights['decoder']))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    init = tf.global_variables_initializer()    
    sess.run(init)
    
    # Train model
    for i in range(num_iters):
        batch_x = data_generator.next()
        _, l = sess.run([optimizer, loss], feed_dict={X: batch_x})
        if i%show_every == 0:
            print("Training loss (MSE) : %.4f" % (l))
    
    # Test model and report NMAE
    test_data_pred = sess.run(decoder_op, feed_dict={X: test_data})
    return nmae(test_data, test_data_pred)

In [29]:
movies = parser.load_data(dd + "u.item", parser.Movie, '|')
ratings = []

for i in range(1,1+5):
    ratings.append( [parser.load_data(dd + "u" + str(i) + ".base", parser.Rating, '\t'), parser.load_data(dd + "u" + str(i) + ".test", parser.Rating, '\t')] )
users = parser.load_data(dd + "u.user", parser.User, '|')

errors = []
for fold in ratings:
    R = parser.R_matrix(len(users), len(movies), fold[0]).astype('float32')
    R_test = parser.R_matrix(len(users), len(movies), fold[1]).astype('float32')
    
    # Fill missing entries with 3, as done in the paper
    R_test += (R_test==0)*3
    
    data_generator = parser.get_next_batch(R, batch_size)
    num_hidden = 1
    #[10, 20, 40, 80, 100, 200, 300, 400, 500]
    
    error = autorec(R.shape[1], 100, data_generator, R_test)
    print("NMAE for this validation: %f" % (error))
    errors.append(error)
    break
# print("NMAE for all folds: %f" % (sum(errors)/float(len(errors))))

Training loss (MSE) : 8419.6084
Training loss (MSE) : 7618.3550
Training loss (MSE) : 6893.1523
Training loss (MSE) : 6237.1587
Training loss (MSE) : 5643.6250
Training loss (MSE) : 5106.8501
Training loss (MSE) : 4620.1230
Training loss (MSE) : 4180.6426
Training loss (MSE) : 3783.2258
Training loss (MSE) : 3422.7029
Training loss (MSE) : 3097.1973
Training loss (MSE) : 2802.4688
Training loss (MSE) : 2535.7634
Training loss (MSE) : 2294.1951
Training loss (MSE) : 2076.0732
Training loss (MSE) : 1878.7706
Training loss (MSE) : 1699.6700
Training loss (MSE) : 1538.0349
Training loss (MSE) : 1391.9501
Training loss (MSE) : 1259.2211
Training loss (MSE) : 1139.6647
Training loss (MSE) : 1031.1251
Training loss (MSE) : 933.0428
Training loss (MSE) : nan
Training loss (MSE) : 764.4184
Training loss (MSE) : 691.2404
Training loss (MSE) : 625.3925
Training loss (MSE) : 566.0485
Training loss (MSE) : 512.3647
Training loss (MSE) : 463.5489
Training loss (MSE) : 419.6977
Training loss (MSE) : 